## Importamos las Librerias a Trabajar 

In [ ]:
import requests
import pandas as pd
from datetime import date,datetime, timedelta
import matplotlib.pyplot as plt
from pansql import sqldf
import teradatasql
import logging
import holidays_co
import os
import sys

tomar la Fecha del sistema con el fin de crear una carpeta de manera local en donde se guardarn el Log y los archivos de Salida

In [ ]:
#The code you provided is creating a folder with a specific name based on the current date and time.
titulo = "Resumen por Fecha"

now = datetime.now()
if now.day<10:
    dia = "0"+str(now.day)
else:
    dia = str(now.day)

if now.month<10:
    mes = "0"+str(now.month)
else:
    mes = str(now.month)

NombreCarpeta = dia + mes + str(now.year)+'_D'

ubicacion_fija = os.getcwd()
ruta_carpeta = os.path.join(ubicacion_fija, NombreCarpeta)


Verificamos que la Carpeta no este creada en la ubicacion deseada de lo contrario la creaara

In [ ]:
#This code is creating a folder (directory) in the current working directory.
ubicacion_fija = os.getcwd()
ruta_carpeta = os.path.join(ubicacion_fija, NombreCarpeta)

if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)
    print(f"Carpeta '{NombreCarpeta}' creada en '{ubicacion_fija}'")
else:
    print(f"La carpeta '{NombreCarpeta}' ya existe en '{ubicacion_fija}'")


Obtenemos los dias feriados de los años 2023 y 2024

In [ ]:
#The code snippet is creating a list of years `[2023, 2024]` and then iterating over each year to retrieve the holidays in Colombia for that year using the `holidays_co.get_colombia_holidays_by_year()` function. The holidays for each year are stored in the `days` list.
years = [2023,2024]
days = []
for i in years:
    days.append(holidays_co.get_colombia_holidays_by_year(i))

holydays = []
for i in range(len(days)):
    for j in range(len(days[i])):
        holydays.append(days[i][j][0]) 

NomArchivo_log =       ruta_carpeta+'\\'+ dia + mes + str(now.year) +"Validacion.log"

Crear una Funcion la cual permitiria la coneccion a la base de datos, para este caso se creo una conexio a la base Denominada Teradata

In [ ]:
#The code defines a function called `conexion` that takes four parameters: `host`, `user`, `password`, and `query`.
def Connection_TeraData(host, user, password, query):
    try:
        conn = teradatasql.connect(host=host, user=user, password=password)       
        df = pd.read_sql(query, conn)
        conn.close()
        return df

    except Exception as e:       
        print(f"Error de conexión: {str(e)}")
        return False

crear una funcion que consume un API de datos Abiertos, pagina publica en la cual encontramos distintos set de datos

In [ ]:
#The `Connection_DatosAbiertos` function is a Python function that connects to a specified URL and retrieves data from an API.
def Connection_DatosAbiertos(star,end, url):
    all_data = []
    start = f"{star}T00:00:00"
    end = f"{end}T23:59:59"
    params = {
    "$limit": 10000000,
    "$offset": 0,
    "$where": f"fecha_corte >= '{start}' and fecha_corte <='{end}'"
    }
    while True:
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Check for any errors in the response
            data = response.json()  # Convert the response to JSON format
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            data = []

        if not data:  # No more data to fetch, break out of the loop
            break
        all_data.extend(data)
        params["$offset"] += len(data)  # Move the offset for the next request
    df = pd.DataFrame(all_data)
    return df

In [ ]:
#The code defines a function called `date_proc` that takes two parameters: `date_sys` and `days_`.
def date_proc(date_sys,days_):
    List_days = []
    list_weekend = []
    contador =1
    while len(List_days)<days_:
        if (date_sys - timedelta(days=contador)).weekday()==5 or (date_sys - timedelta(days=contador)).weekday()==6:
            list_weekend.append(date_sys - timedelta(days=contador))
        elif (date_sys - timedelta(days=contador)) in holydays:
            list_weekend.append(date_sys - timedelta(days=contador))
        else:
            List_days.append(date_sys - timedelta(days=contador))
        contador = contador + 1
    return min(List_days)

In [ ]:
#The code snippet is calculating the start and end dates based on the current date.
date_prod_i = date_proc(datetime.now(),2)
date_prod_f = date_proc(datetime.now(),3)

if (date_prod_i -date_prod_f).days >1:
    star = (date_prod_f +timedelta(days=1))
    end = date_prod_i
else:
    star = date_prod_i
    end = date_prod_i

star = date(star.year,star.month,star.day).strftime('%Y-%m-%d')
end = date(end.year,end.month,end.day).strftime('%Y-%m-%d')

In [ ]:
url = "https://www.datos.gov.co/resource/qhpu-8ixx.json"
hosttd="11.11.11.111"  #hostname 
userdb="---_------" #usuario de base de datos
pss = '----_------'
query = f"""SELECT
    campo1,
    campo2
    from (select 
            campo1,
            campo2,
            campo3
        From Tabla1
        where campo1 in (1,2,3))as Tbl t inner join Tabla2 b
    on t.campo1 = b.campo1
    and campo2 = b.campo2
    where 
    t.campo between '{star}' and '{end}'
    """

In [ ]:
df_Teradata = Connection_TeraData(hosttd,userdb,pss,query)
logging.info(f"Cargue exitoso de informacion TeraData")
df_datosAbiertos = Connection_DatosAbiertos(star,end,url)
logging.info(f"Cargue exitoso de informacion de Datos Abiertos")
logging.info(f"Fecha Proceso: {datetime.now()}")
logging.info(f"Fecha Corte  : {star}")

In [ ]:
if df_datosAbiertos.shape[0]==0:
    logging.warning(f"No hay datos Cargados para la fecha de corte {star}")
    logging.warning(f"Finaliza Proceso")
    sys.exit()
else:
    Columns_df = list(df_datosAbiertos.columns)
    Columns_TeraData = list(df_Teradata.columns)

    Columns_df_m = []
    Columns_TeraDta_m = []

    for i in Columns_df:
        Columns_df_m.append(i.lower())

    for i in Columns_TeraData:
        Columns_TeraDta_m.append(i.lower())

    df_datosAbiertos.columns = Columns_df_m
    df_Teradata.columns = Columns_TeraDta_m

    Change_dtypes_TeraData= {
    
    "campo1":"uint16",
    "campo2":"category",
    "Campo3":"category",
    "Campo4":"category",
    "Campo5":"uint8",
    "Campo6":"object",
    "Campo7":"uint16",
    "Campo8":"uint8",
    "Campo9":"object",
    "Campo0":"uint8",
    "Campo10":"category",
    "Campo11":"category",
    "Campo12":"category",
    "Campo13":"uint8",
    "Campo14":"category",
    "Campo15":"uint8",
    "Campo16":"category",
    "Campo17":"category",
    "Campo18":"uint32",
    "Campo19":"object",
    "Campo20":"uint8",
    "Campo21":"uint16",
    "Campo22":"category"
    }

    Change_dtypes_DatosAbiertos = {
    "tipo_entidad":"category",
    "nombre_tipo_entidad":"category",
    "codigo_entidad":"uint8",
    "nombre_entidad":"object",
    "tipo_negocio":"uint8",
    "nombre_tipo_patrimonio":"object",
    "subtipo_negocio":"uint8",
    "nombre_subtipo_patrimonio":"category",
    "codigo_negocio":"uint32",
    "nombre_patrimonio":"object",
    "principal_compartimento":"uint8",
    "tipo_participacion":"uint16"

    }

    df_Teradata.campo1 = pd.to_datetime(df_Teradata.campo1, format='%d/%m/%Y')
    df_datosAbiertos.fecha_corte = pd.to_datetime(df_datosAbiertos.fecha_corte)#, format='%d/%m/%Y')
    df_Teradata.campo10 = pd.to_datetime(df_Teradata.campo10,format = '%d/%m/%Y')

    for i in Change_dtypes_DatosAbiertos.keys():
        df_datosAbiertos[i.lower()] = df_datosAbiertos[i].astype(Change_dtypes_DatosAbiertos[i])


    for i in Change_dtypes_TeraData.keys():
        df_Teradata[i] = df_Teradata[i].astype(Change_dtypes_TeraData[i])

    base_date = datetime(1900, 1, 1)

    def fecha_a_numero_serie(fecha):
        diferencia = (fecha - base_date).days
        return diferencia+2

    df_Teradata["Mes"] = (df_Teradata["campo1"].dt.month).astype("uint8")
    df_Teradata["Anho"] = (df_Teradata["campo1"].dt.year).astype("uint16")
    df_Teradata["Fecha_num"] = (df_Teradata["campo1"].apply(fecha_a_numero_serie)).astype("uint16")
    df_Teradata["fecha_corte"] = df_Teradata["campo1"].dt.strftime('%d/%m/%Y')

    df_datosAbiertos["Mes"] = df_datosAbiertos.fecha_corte.dt.month
    df_datosAbiertos["Anho"] = df_datosAbiertos.fecha_corte.dt.year
    df_datosAbiertos["Mes"] = df_datosAbiertos["Mes"].astype("uint8")
    df_datosAbiertos["Anho"] = df_datosAbiertos["Anho"].astype("uint16")
    df_datosAbiertos["Fecha_num"] = (df_datosAbiertos["fecha_corte"].apply(fecha_a_numero_serie)).astype("uint16")
    df_datosAbiertos["Fecha_num"] = df_datosAbiertos["Fecha_num"].astype("str")

In [ ]:
#Creamos los DataFrame con la primary Key para realizar el cruce correspondiente
Df_T = df_Teradata[['Key',"fecha_corte"]]
Df_D = df_datosAbiertos[['Key','fecha_corte']]

#realizamos el cruce correspondiente entre los Dataframe para evidenciar lo que se esta registrado en TeraData y no esta en Datos Abiertos
TDTA_vs_DA = pd.merge(left = Df_T,
                         right = Df_D, 
                         on='Key', 
                         how='left',
                         suffixes = ('_T','_DA'))

#realizamos el cruce correspondiente entre los Dataframe para evidenciar lo que se esta registrado en Datos Abiertos y no esta en TeraData
DA_vs_TDTA = pd.merge(left =Df_T,
                         right = Df_D, 
                         on='Key', 
                         how='right',
                         suffixes = ('_DA','_T'))

#filtramos los resultados por los datos nulos
TDTA_vs_DA = TDTA_vs_DA[TDTA_vs_DA['fecha_corte_DA'].isnull()]
DA_vs_TDTA = DA_vs_TDTA[DA_vs_TDTA["fecha_corte_T"].isnull()]

#eliminamos los campos de fecha para solo dejar la llave primaria 
TDTA_vs_DA = TDTA_vs_DA.drop(columns = ['fecha_corte_DA','fecha_corte_T'])
DA_vs_TDTA = DA_vs_TDTA.drop(columns = ['fecha_corte_DA','fecha_corte_T'])

#cruazamos los resultados con las tablas originales para crear las tablas de salida
df_sin_trasmitir_DA = pd.merge(left = df_Teradata,
                         right = TDTA_vs_DA, 
                         on='Key', 
                         how='inner',
                         suffixes = ('_T','_C'))

df_sin_trasmitir_TD = pd.merge(left = df_datosAbiertos,
                         right = DA_vs_TDTA, 
                         on='Key', 
                         how='inner',
                         suffixes = ('_DA','_C'))
#Creamos la columna en la cual se ejecuta el proceso 
df_sin_trasmitir_DA["Fecha_proceso"] = datetime.now()
df_sin_trasmitir_TD["Fecha_Proceso"] = datetime.now()

logging.info(f'Registros en TeraData y no cargados en Datos Abiertos {df_sin_trasmitir_DA.shape[0]}')
logging.info(f'Registros en Datos Abiertos y no Registrados en TeraData {df_sin_trasmitir_DA.shape[0]}')
#exportamos los datos a un txt
NombreArchivo1 = ruta_carpeta+'\\'+ mes + dia + str(datetime.now().year) +'_Incremetal_sin_trasmitir_DatosAbiertos.txt' 
NombreArchivo2 = ruta_carpeta+'\\'+ mes + dia + str(datetime.now().year) +'_Incremetal_sin_registro_TeraData.txt'
logging.info(f'Archivo con diferencias se ejecto correctamente, Nombre del Archivo {NombreArchivo1}')
logging.info(f'Archivo con diferencias se ejecto correctamente, Nombre del Archivo {NombreArchivo2}')
df_sin_trasmitir_DA.to_csv(NombreArchivo1, sep='|', index=False)
df_sin_trasmitir_TD.to_csv(NombreArchivo2, sep='|', index=False)